In [1]:
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix as cm


In [2]:
def get_data(path):
    df = pd.read_excel(path)
    cols = list(df.columns)
    col_x = cols[1:-1]
    col_y = cols[-1]
    mu = df[col_x].mean(axis=0)
    std = df[col_x].std(axis=0)
    df[col_x] = (df[col_x]-mu)/std
    X = df[col_x].values
    Y = df[col_y]
    names = list(set(Y))
    K = len(names)
    N = len(Y)
    namestoint = dict({name:i for i,name in enumerate(names)})
    T = np.zeros((N, K))
    for i in range(N):
        T[i, namestoint[Y[i]]] = 1
    return X,T

In [3]:
path = r"C:\Users\gprak\Downloads\BITS Courses\CS F464 Machine Learning I\data\data5.xlsx"
X,Y = get_data(path)
X,Y = shuffle(X,Y)
train_split = 0.7
split_idx = int(train_split*len(X))
X_train = X[:split_idx]
Y_train = Y[:split_idx]
X_test = X[split_idx:]
Y_test = Y[split_idx:]


In [4]:
def k_means(X,Y,K,max_iters=20):
    N,D = X.shape
#     K = len(np.unique(Y,axis=0))
    cluster_identities = np.zeros(N)
    cluster_centers = np.zeros((K,D))
    for k in range(K):
        i = np.random.choice(N)
        cluster_centers[k] = X[i]
    min_dists = np.zeros(N)
    costs = []
    for i in range(max_iters):
        old_cluster_identities = cluster_identities.copy()
        for n in range(N):
            closest_k = -1
            min_dist = float('inf')
            for k in range(K):
                d = (X[n]-cluster_centers[k]).dot(X[n]-cluster_centers[k])
                if d < min_dist:
                    min_dist = d
                    closest_k = k
            cluster_identities[n] = closest_k
            min_dists[n] = min_dist
        costs.append(min_dists.sum())
#         print(sorted(np.unique(cluster_identities)))
        for k in range(K):
#             print(len(X[cluster_identities==k]))
            if(len(X[cluster_identities==k]))==0:
                cluster_centers[k,:]=0
            else:
                cluster_centers[k,:] = X[cluster_identities==k].mean(axis = 0)
        if np.all(old_cluster_identities==cluster_identities):
            print(f"Converged on step {i}")
            break
    return costs,cluster_centers,cluster_identities

In [5]:
K = 600
costs,c,labels = k_means(X_train,Y_train,K)

Converged on step 5


In [6]:
# cluster_centers

In [7]:
# kmeans = KMeans(n_clusters=K,random_state=0).fit(X_train)

In [8]:
# labels=kmeans.labels_
# labels

In [9]:
# c = kmeans.cluster_centers_
# c

In [10]:
n = np.zeros(K)

In [11]:

for i in range(K):
    n[i] = (labels==i).sum()

In [12]:
sigma = np.zeros(K)

In [13]:
for i in range(K):
    sigma[i] = (X_train[labels==i,:]-c[i]).T.dot(X_train[labels==i,:]-c[i]).sum()/n[i]

C:\Users\gprak\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


In [14]:
sigma = np.nan_to_num(sigma)
sigma

array([9.38592853e+01, 1.19594722e+02, 1.40465115e+02, 1.03790427e+01,
       5.74736766e+00, 5.09921058e+00, 1.13480868e+01, 7.40257128e+00,
       1.48564974e+00, 0.00000000e+00, 1.89469438e+00, 0.00000000e+00,
       0.00000000e+00, 9.40364913e+00, 8.63417372e+00, 2.14706641e+01,
       4.10062454e-01, 3.40597134e+01, 8.82163187e+00, 2.02855703e+01,
       5.28537602e+01, 1.67532763e+01, 0.00000000e+00, 0.00000000e+00,
       1.44362643e+00, 9.43567929e+00, 0.00000000e+00, 1.86163308e-02,
       4.74705494e+01, 2.24472589e+00, 0.00000000e+00, 1.76603325e+01,
       9.22694952e+00, 1.80523639e+01, 3.24813920e+01, 1.56721422e+00,
       0.00000000e+00, 0.00000000e+00, 7.44414901e+00, 1.33137916e+02,
       2.77956937e+01, 0.00000000e+00, 0.00000000e+00, 4.48748181e-02,
       9.17625277e-02, 8.30808340e+01, 0.00000000e+00, 5.10451972e+00,
       1.07250869e-02, 1.53413279e+01, 0.00000000e+00, 0.00000000e+00,
       6.21789803e-01, 0.00000000e+00, 1.22351808e+00, 0.00000000e+00,
      

In [15]:
beta = 0.5*sigma*sigma


In [16]:
H = np.zeros((len(X_train),len(c)))

H.shape

(1503, 600)

In [17]:
for i in range(len(X_train)):
    for j in range(len(c)):
        H[i,j] = np.exp(-beta[j]*(X_train[i,:]-c[j,:]).dot(X_train[i,:]-c[j,:]).sum())

In [18]:
W = np.linalg.pinv(H).dot(Y_train)
W

array([[-1.82811057e+007,  1.82811057e+007],
       [ 5.05983180e+007, -5.05983180e+007],
       [-3.71536994e+007,  3.71536994e+007],
       ...,
       [-5.61050860e+005,  5.61050865e+005],
       [ 5.98568866e-131, -5.98568866e-131],
       [ 2.35121109e+006, -2.35121108e+006]])

In [19]:
Ht = np.zeros((len(X_test),len(c)))
Ht.shape     

(645, 600)

In [20]:
for i in range(len(X_test)):
    for j in range(len(c)):
        Ht[i,j] = np.exp(-beta[j]*(X_test[i,:]-c[j,:]).dot(X_test[i,:]-c[j,:]).sum())


In [21]:
yp = Ht.dot(W)
yp.shape

(645, 2)

In [22]:
yp = np.round(yp)
yp

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [23]:
def get_metrics(y_true,y_pred):
    confusion_matrix = cm(y_true,y_pred)
    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.sum() - (FP + FN + TP)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)

    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    return {
        "ACC":ACC,
        "TPR":TPR,
        "TNR":TNR
    }

In [24]:
y_true = []
for i in range(len(Y_test)):
    if Y_test[i,0]>Y_test[i,1]:
        y_true.append(0)
    else:
        y_true.append(1)

In [25]:
y_pred = []
for i in range(len(yp)):
    if yp[i,0]>yp[i,1]:
        y_pred.append(0)
    else:
        y_pred.append(1)

In [26]:
get_metrics(y_true,y_pred)

{'ACC': array([0.81550388, 0.81550388]),
 'TPR': array([0.8    , 0.83125]),
 'TNR': array([0.83125, 0.8    ])}